# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,23.99,94,75,5.14,CC,1722813895
1,1,hermanus,-34.4187,19.2345,10.05,65,80,6.76,ZA,1722813607
2,2,adamstown,-25.0660,-130.1015,21.30,78,91,7.87,PN,1722813897
3,3,stanley,54.8680,-1.6985,15.40,92,40,3.09,GB,1722813898
4,4,dzhalagash,45.0833,64.6667,24.88,28,1,2.78,KZ,1722813899


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.6,
    color = "City",
    frame_width = 800

    
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
#Tempature
ideal_vacation_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 18)]
#Wind Speed
ideal_vacation_df = city_data_df.loc[city_data_df["Wind Speed"] < 2]
#Cloudiness
ideal_vacation_df = city_data_df.loc[city_data_df["Cloudiness"] < 20]
# Drop any rows with null values
ideal_vacation_df = ideal_vacation_df.dropna()

# Display sample data
ideal_vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,dzhalagash,45.0833,64.6667,24.88,28,1,2.78,KZ,1722813899
9,9,port elizabeth,-33.9180,25.5701,11.16,62,0,7.20,ZA,1722813905
12,12,igarka,67.4667,86.5833,7.22,73,2,6.03,RU,1722813908
21,21,navasota,30.3880,-96.0877,35.62,60,10,3.84,US,1722813691
23,23,badger,64.8000,-147.5333,24.09,53,0,2.57,US,1722813922


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame.copy(ideal_vacation_df[["City","Lat","Lng","Country","Humidity"]])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Country,Humidity,Hotel Name
4,dzhalagash,45.0833,64.6667,KZ,28,
9,port elizabeth,-33.9180,25.5701,ZA,62,
12,igarka,67.4667,86.5833,RU,73,
21,navasota,30.3880,-96.0877,US,60,
23,badger,64.8000,-147.5333,US,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "limit":1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dzhalagash - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
igarka - nearest hotel: Заполярье
navasota - nearest hotel: Days Inn
badger - nearest hotel: Clarion Hotel & Suites
avarua - nearest hotel: Paradise Inn
ribeira grande - nearest hotel: Hospedaria JSF
namibe - nearest hotel: Nelsal Pensao
dalmatovo - nearest hotel: Старт
qaqortoq - nearest hotel: Hotel Qaqortoq
strugi-krasnyye - nearest hotel: МП ЖКХ
sitka - nearest hotel: Aspen Suites Hotel
skadovs'k - nearest hotel: Затишний
mossel bay - nearest hotel: Protea Hotel Mossel Bay
mwinilunga - nearest hotel: Guest House le Manguier
waddan - nearest hotel: فندق الواحة-هون
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
ust'-abakan - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
anatuya - nearest hotel: Hotel Santa Rita
beas de segura - nearest hotel: Hotel Avenida
taoudenni - nearest hotel: فندق السايح
remire-montjoly - nearest hotel:

,City,Lat,Lng,Country,Humidity,Hotel Name
4,dzhalagash,45.0833,64.6667,KZ,28,No hotel found
9,port elizabeth,-33.9180,25.5701,ZA,62,Waterford Hotel
12,igarka,67.4667,86.5833,RU,73,Заполярье
21,navasota,30.3880,-96.0877,US,60,Days Inn
23,badger,64.8000,-147.5333,US,53,Clarion Hotel & Suites
...,...,...,...,...,...,...
557,ingeniero guillermo n. juarez,-23.9000,-61.8500,AR,54,Hotel El Indiano
559,aoulef,26.9667,1.0833,DZ,10,دار الضياف
560,taman',45.2117,36.7161,RU,78,Волна
564,hammerfest,70.6634,23.6821,NO,68,Thon Hotel Hammerfest


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_map =  hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.6,
    color = "City",
    frame_width = 800,
    hover_cols=["Hotel Name","Country"]
    
    
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)